In [1]:
from utils import *
from config import *

import numpy as np 
import math 


# Load data

In [4]:
connector = Connector(**BIGQUERY_CONFIG)


istanbul_inputs = {
    "shape_query" : QueryArgs(
        filename = "area-shapes.sql",
        params = {
            "country_code":"tr",
            "city":"Istanbul"
        }
    )

    , "session_query" : QueryArgs(
        filename = "sessions-with-area.sql",
        params = {
            "country_code":"tr",
            "city":"Istanbul",
            "entity_id":"YS_TR",
            "weeks_ago":3,
            "city_id":1,
            "asa_ids":[334]
        }
    )

    , "city_query" : QueryArgs(
        filename = "city-dh-shape.sql",
        params = {
            "entity_id":"YS_TR",
            "city_id":1
        }
    )
}

In [7]:
session_data = connector.get_df_from_query(QueryHandler.build_query(istanbul_inputs["session_query"]))

area_shapes = (
    connector
    .get_df_from_query(QueryHandler.build_query(istanbul_inputs["shape_query"]))
    .pipe(convert_to_geopandas, "area_shape")
    .drop(columns=["area_shape"])
)

city_dh_shape = (
    connector
    .get_df_from_query(QueryHandler.build_query(istanbul_inputs["city_query"]))
    .pipe(convert_to_geopandas, "city_shape")
    .drop(columns=["city_shape"])
)

Job ID 0d138cd8-3043-4f14-b7ce-6c56145293e1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Job ID 01a2ab2c-bf88-4207-82e0-ba6a9ecb305b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Job ID 5185dbfa-7009-490a-97c5-1de6579360c6 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [8]:
session_data.groupby("area_name").size().sort_values(ascending=False)
filter_area = "Kadıköy"
test_area = session_data.query(f"area_name=='{filter_area}'")

(
    test_area 
    .assign(
        cvr_user = lambda df: df.n_conversions.div(df.n_sessions)
    )
    .agg(
        total_sessions = ("n_sessions", "sum"),
        total_conversions = ("n_conversions", "sum"),
        n_users = ("perseus_client_id", "count"),
        avg_cvr = ("cvr_user", "mean")
    )
    # .melt()
    # .dropna()
)

,n_sessions,n_conversions,perseus_client_id,cvr_user
total_sessions,968047.0,NaN,NaN,NaN
total_conversions,NaN,39989.0,NaN,NaN
n_users,NaN,NaN,198673.0,NaN
avg_cvr,NaN,NaN,NaN,0.041348


# Code

In [9]:

@dataclass
class DeltaParameters:
    """Holds the parameters relevant to delta method variance.
    more info here: https://arxiv.org/abs/2305.16459

    As a general definition
    denom_mean -> Denominator mean, 
    demon_var -> Denominator Variance
    num_mean -> numerator mean
    num_var -> numerator variance
    covar -> covariance between numerator and denominator

    For example, sessions per user acts a denominator while transactions per user does it as numerator.
    """
    denom_mean:float
    num_mean:float 
    denom_var:float 
    num_var:float 
    covar:float 
    cvr:float 
    sample_size:float
    
    def calculate_variance_factor(self):
        """ h is the variable name in the paper source. 
        I view is as an adjusted variance. This is a separated
        function as this factor is useful for both the design and analysis phase. 

        Returns:
            _type_: _description_
        """
        num = self.num_var - 2*self.covar*self.num_mean/self.denom_mean + (self.num_mean**2)/(self.denom_mean**2)*self.denom_var
        den = self.denom_mean**2
        return num/den
    
    def calculate_sample_variance(self):
        """Full variance calculation requires sample size.
        Returns:
            _type_: _description_
        """
        return self.calculate_variance_factor() / self.sample_size
    
@dataclass
class PowerParams:
    """Class to hold default inputs for power analysis. 
    As a note, MDE must be ABSOLUTE
    """
    mde:float 
    alpha:float = 0.05
    power:float = 0.8
    n_variants:int=2

    def calculate_z_params(self) -> tuple:
        """Return the equivalent z-score
        of the alpha and power params.
        It assumes a two-tails test by default.

        Returns:
            _type_: _description_
        """
        z_alpha = norm.ppf(1-self.alpha/2)
        z_beta = norm.ppf(self.power)
        return (z_alpha, z_beta)
    

def aggregate_ratio_data(df:pd.DataFrame, denom_col:str, num_col:str) -> DeltaParameters:
    """Take a dataframe that holds information at the randomization level (e.g user) and 
    calculate the variance/averages parameters used in the delta method.

    Args:
        df (pd.DataFrame): _description_
        session_col (str): _description_
        conversion_col (str): _description_

    Returns:
        DeltaParameters: _description_
    """
    params =  {
            "denom_mean": df[denom_col].mean()
            , "num_mean": df[num_col].mean()
            , "denom_var": df[denom_col].var()
            , "num_var":df[num_col].var()
            , "covar":df[denom_col].cov(df[num_col])
            , "cvr": df[num_col].sum() / df[denom_col].sum()
            , "sample_size": df.shape[0]
        }
    return DeltaParameters(**params)

def calculate_delta_method_sample_size(delta_params: DeltaParameters, power_inputs:PowerParams) -> float:
    """Calculate number of users (sample size) using the delta method.
    Method details are in this paper: https://arxiv.org/abs/2305.16459

    As a summary, the delta method uses a corrected formula to get
    the correct variance in the case sessions are correlated. This correlation usually happens
    experiment randomizes on users but CVR is on session level.

    Args:
        delta_params (DeltaParameters): _description_
        power_inputs (PowerInputs): _description_

    Returns:
        float: _description_
    """
    
    h = delta_params.calculate_variance_factor()
    z_alpha, z_power = power_inputs.calculate_z_params()
    k = math.ceil ( ( 2 * h * ( (z_alpha + z_power)**2)) / (power_inputs.mde ** 2) )
    return k


In [10]:
@dataclass
class TestResult:
    statistic:str
    statistic_val:float
    p_val:float 
    is_significant:bool 
    alpha:float


def variant_name_generator(n_variants:int) -> list[str]:
    """Generate a list of variant names starting from "Control"
    up to VariantX where X is the number of variants to use
    in an Experiment

    Args:
        n_variants (int): Number of variants (including Control)

    Returns:
        list[str]: _description_
    """
    return ["Control"] + [f"Variation{x}" for x in range(1, n_variants)]


def allocate_into_variants(df:pd.DataFrame, n_variants:int) -> pd.DataFrame:
    """

    Add a new column called "test_variant" in which rows are assigned to
    one of the Variants

    Args:
        df (pd.DataFrame): _description_
        n_variants (int): _description_

    Returns:
        pd.DataFrame: _description_
    """
    variant_list = variant_name_generator(n_variants)
    return (
        df 
        .assign(
            test_variant = np.random.choice(variant_list, df.shape[0])
        )
    )

def split_into_a_b(df:pd.DataFrame, variant_a:str, variant_b:str):
    """Simply split a single dataframe into two variants. 
    As a future improvement, support split into a/b/n. 
    Args:
        df (pd.DataFrame): _description_
        variant_a (str): _description_
        variant_b (str): _description_

    Returns:
        _type_: _description_
    """
    df_var_a = df.query("test_variant==@variant_a") 
    df_var_b = df.query("test_variant==@variant_b")
    return df_var_a, df_var_b


def apply_delta_method_test(var_a:pd.DataFrame, var_b:pd.DataFrame, denom_col:str, num_col:str, alpha:float=0.05) -> bool:
    """This function applies hypothesis testing following the delta method for ratio metric.
    In a nutshell, once the corrected variance has been calculated, estimate the z-score
    and subsequent p-value.


    Args:
        var_a (pd.DataFrame): _description_
        var_b (pd.DataFrame): _description_
        denom_col (str): _description_
        num_col (str): _description_
        alpha (float, optional): _description_. Defaults to 0.05.

    Returns:
        bool: _description_
    """

    # aggregate input data
    agg_var_a = var_a.pipe(aggregate_ratio_data, denom_col, num_col)
    agg_var_b = var_b.pipe(aggregate_ratio_data, denom_col, num_col)

    # simulate h0
    cvr_diff = agg_var_b.cvr - agg_var_a.cvr
    var_sum = agg_var_b.calculate_sample_variance() + agg_var_a.calculate_sample_variance()

    # calculate p-value
    z_score = cvr_diff / np.sqrt(var_sum)
    p_val = min(norm.cdf(z_score), 1-norm.cdf(z_score))
    is_significant = p_val < alpha / 2

    return TestResult(
        statistic="z-test",
        statistic_val=z_score,
        p_val=p_val, 
        is_significant = is_significant,
        alpha = alpha / 2
    )





# Formula Sample size 

In [11]:
power_params = PowerParams(mde=0.01, n_variants=2)
delta_params = aggregate_ratio_data(test_area, "n_sessions", "n_conversions")
size_per_variant = calculate_delta_method_sample_size(delta_params, power_params)
size_per_variant

3654

# Simulation

Weight is used to easily modify the sample size to take from the dataframe. This is if i want to take lower or higher sample sizes than what the formula says to check if simulation validates the sample size output 

In [23]:
weight = 1.5

sample_df = test_area.sample(
    n = int(weight*size_per_variant*power_params.n_variants)
    , replace= False
)

sample_df.head()

,city,area_name,perseus_client_id,n_sessions,n_conversions
812826,Istanbul,Kadıköy,1699548984594.761799240117872985.ZSoHiIJlQR,1,0
1338236,Istanbul,Kadıköy,1661190068086.921797088482393544.C5eok0SEMH,1,0
2446559,Istanbul,Kadıköy,1677414917789.0840827516.tnowdnsboe,9,0
285857,Istanbul,Kadıköy,1671464980063.4701156324.nnixhfsiou,1,0
1871687,Istanbul,Kadıköy,1676534004157.5129371850.hioemrjlpj,9,1


## A/A 

In [24]:
from tqdm import tqdm

results_AA = []
for it in tqdm(range(1000)):

    it_df = sample_df.sample(frac=1, replace=True)

    control, variation = (
        it_df
        .pipe(allocate_into_variants, 2)
        .pipe(split_into_a_b, "Control", "Variation1")
    )

    significant = apply_delta_method_test(
        var_a = control
        , var_b = variation
        , denom_col="n_sessions"
        , num_col="n_conversions"
        , alpha=power_params.alpha

    )

    results_AA.append(significant)

# np.mean(results_AA)

100%|██████████| 1000/1000 [00:17<00:00, 57.81it/s]


## A/B

Difficulty here is how to correctly apply the treatment as a noise. 


In [25]:

def add_noise(variation:pd.DataFrame, mde:float, denom_col:str, num_col:str) -> pd.DataFrame:
    """Simulate the treatment effect. The key is to model the target metric
    distribution. This method is based on the BinomialNoiser class
    from: https://github.com/deliveryhero/logistics-ds-location-researches/blob/main/choice_ab_test_analysis/src/basic_noisers.py

    The logic is to view use the number of session as variable to cohort the users. Users who over a period
    of time has done the same number of session, their conversion behaviour should also be similar, on average.
    The treatment effect is added to this cohort behaviour.

    another option, not implemented here, is to fit a distribution of the CVR by user and draw a sample
    for each user. This is what they did in the reference paper https://arxiv.org/abs/2305.16459. The reason I
    didn't implement it here is that I have a highly skew distribution with many user with 0 CVR which requires more research into which
    distribution I can use. In the paper they use a normal for the simulation which is not applicable to this context.
    
    Args:
        variation (pd.DataFrame): _description_
        mde (float): _description_
        denom_col (str): _description_
        num_col (str): _description_

    Returns:
        pd.DataFrame: _description_
    """
    bucket_df = (
        variation
        .assign(
            bucket_cvr = variation[num_col] / variation[denom_col]
        )    .groupby(denom_col, as_index=False)
        ["bucket_cvr"]
        .mean()
    )


    return (
        variation
        .drop(columns=[num_col])
        .merge(bucket_df, left_on=[denom_col], right_on=[denom_col])
        .assign(**{
            num_col: lambda df: np.random.binomial(
                n=df[denom_col] 
            , p=(df["bucket_cvr"] + mde).clip(0,1) 
            )}
        )
    )



In [26]:
results_AB = []
for it in tqdm(range(1000)):

    it_df = sample_df.sample(frac=1, replace=True)

    control, variation = (
        it_df
        .pipe(allocate_into_variants, 2)
        .pipe(split_into_a_b, "Control", "Variation1")
    )

    variation_adj = add_noise(variation, power_params.mde, "n_sessions", "n_conversions")

    significant = apply_delta_method_test(
        var_a = control
        , var_b = variation_adj
        , denom_col="n_sessions"
        , num_col="n_conversions"
        , alpha=power_params.alpha

    )

    results_AB.append(significant)


100%|██████████| 1000/1000 [00:22<00:00, 44.75it/s]


In [27]:
np.mean([x.is_significant for x in results_AA]), np.mean([x.is_significant for x in results_AB])

(0.048, 0.935)